In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np
import importlib as imp

from collections import namedtuple
from random import sample, shuffle
from functools import reduce
from itertools import accumulate
from math import floor, ceil, sqrt, log, pi
from matplotlib import pyplot as plt
from tensorflow.keras import layers, utils, losses, models as mds, optimizers

if imp.util.find_spec('aggdraw'): import aggdraw
if imp.util.find_spec('tensorflow_addons'): from tensorflow_addons import layers as tfa_layers
if imp.util.find_spec('tensorflow_models'): from official.vision.beta.ops import augment as visaugment
if imp.util.find_spec('tensorflow_probability'): from tensorflow_probability import distributions as tfd

2022-03-28 14:59:02.181001: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-03-28 14:59:02.181083: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [ ]:
def mean_per_class_acc(y_true, y_pred):
    score, up_opt = tf.compat.v1.metrics.mean_per_class_accuracy(y_true, y_pred, num_classes=2)
    tf.print(score.eval())

    score = tf.identity(score)       
    return score

# metric = tf.compat.v1.metrics.mean_per_class_accuracy([1, 0], [[.2, .8], [.1, .9]], 2)
with tf.compat.v1.Session() as sess:
    y_true = tf.one_hot([1, 0], 2)
    y_pred = tf.constant([[.2, .8], [.1, .9]], shape=(2,2))
    score = mean_per_class_acc(y_true, y_pred)
    print(score)

# Signed IoU Metric (Sparse Variant)

* It produces a positive value for overlapping and a negative value for non-overlapping boxes.


In [ ]:
IOU_EPSILON = 0.0000000000001/(IMG_SIZE*IMG_SIZE)

def compute_signed_iou(boxes_1, boxes_2):
    """
        It computes the IoU of boxes in boxes_1 with the corresponding boxes in boxes_2.
        A negative IoU is a measure of the non-overlap between the corresponding boxes.

        Arguments:
            boxes_1: A tensor of boxes with shape (N_BOXES, 4) in YXYX format.
            boxes_2: A tensor of boxes with shape (N_BOXES, 4) in YXYX format.

        Returns:
            * Intersections of boxes in boxes_1 and boxes_2 in YXHW format. Shape: (N_BOXES, 4)
            * IoUs of shape (N_BOXES, 1)
    """
    [b1_yx_min, b1_yx_max] = tf.split(boxes_1, 2, axis=-1)
    [b2_yx_min, b2_yx_max] = tf.split(boxes_2, 2, axis=-1)

    # Pick maximum yx_min and minimum yx_max to get the intersection
    yx_min = tf.where(b1_yx_min > b2_yx_min, b1_yx_min, b2_yx_min)
    yx_max = tf.where(b1_yx_max < b2_yx_max, b1_yx_max, b2_yx_max)

    intersection_yxyx = tf.concat([yx_min, yx_max], axis=-1)
    intersection_yxhw = yxyx_to_yxhw(intersection_yxyx)

    # Compute intersection height and width
    y_min, x_min = tf.split(yx_min, 2, axis=-1)
    y_max, x_max = tf.split(yx_max, 2, axis=-1)
    intersection_h, intersection_w = y_max - y_min, x_max - x_min

    # Record negative heights and widths. We'll force a negative area
    # if both of them are negative.
    negative_h_intersection = intersection_h < 0
    negative_w_intersection = intersection_w < 0
    negative_hw_intersection = tf.math.logical_and(negative_h_intersection, negative_w_intersection)

    # Compute intersection and union and use them to compute
    # IoUs
    intersection = tf.where(negative_hw_intersection, intersection_h*-intersection_w, intersection_h*intersection_w)
    b1_area, b2_area = box_area(boxes_1), box_area(boxes_2)
    union = b1_area + b2_area - intersection
    iou = intersection/(union + IOU_EPSILON)

    return intersection_yxhw, iou

def evaluate_boxes(boxes, predictions):
    """
        It computes the IoU of boxes and predictions using compute_signed_iou() function

        Arguments:
            boxes: A tensor of boxes with shape (N_BOXES, 4) in YXYX format.
            predictions: A tensor of boxes with shape (IMG_SIZE, IMG_SIZE, 2) in HW format.

        Returns:
            * A tensor of predictions in YXHW format. Shape (N_BOXES, 4)
            * Intersections of boxes with predictions in YXHW format. Shape: (N_BOXES, 4)
            * IoUs of shape (N_BOXES, 1)
    """
    boxes = boxes.to_tensor()
    size = predictions.shape[0]

    boxes_yx_min = boxes[:, :2]
    yx_min_indices = yx_to_indices(boxes_yx_min, size)

    # tf.print('boxes_yx_min: ', boxes_yx_min, boxes_yx_min.shape)
    # tf.print('yx_min_indices: ', yx_min_indices, yx_min_indices.shape)

    hw_preds = tf.gather_nd(predictions, yx_min_indices)[:, :2]
    yxhw_preds = tf.concat([boxes_yx_min, hw_preds], axis=-1)
    yxyx_preds = yxhw_to_yxyx(yxhw_preds)

    # tf.print('hw_preds: ', hw_preds, hw_preds.shape)
    # tf.print('yxhw_preds: ', yxhw_preds, yxhw_preds.shape)

    yxhw_intersection, iou = compute_signed_iou(boxes, yxyx_preds)

    # return yxhw_preds, yxhw_intersection, tf.reduce_mean(iou)
    return yxhw_preds, yxhw_intersection, iou

def evaluate_predictions(y_true, y_pred):
    """
        It computes mean IoUs for a batch.

        Arguments:
            y_true: A ragged tensor of true boxes in YXYX format. Shape (BATCH_SIZE, N_BOXES*, 4)
            y_pred: A tensor of predicted boxes in HW format. Shape (BATCH_SIZE, IMG_SIZE, IMG_SIZE, 2)

        Returns:
            * A ragged tensor of predictions in YXHW format. Shape (BATCH_SIZE, N_BOXES*, 4)
            * Intersections of boxes with predictions in YXHW format. Shape: (BATCH_SIZE, N_BOXES*, 4)
            * Mean IoU of shape (1)

    """
    batch_size = tf.shape(y_true)[0]

    yxhw_intersections = []
    yxhw_preds = []
    ious = []

    for item_id in range(batch_size):
    # item_id = 0
        yxhw_pred, yxhw_intersection, iou = evaluate_boxes(y_true[item_id], y_pred[item_id])
        
        yxhw_intersections.append(yxhw_intersection)
        yxhw_preds.append(yxhw_pred)
        ious.append(iou)
    
    mean_iou = tf.reduce_mean(tf.concat(ious, axis=0))

    # tf.print('mean_iou: ', mean_iou)
    
    return yxhw_preds, yxhw_intersections, mean_iou

# Mean Metrics

In [ ]:
class Mean(metrics.Metric):
    def __init__(self, name="mean", **kwargs):
        super(Mean, self).__init__(name=name, **kwargs)
        self.total = self.add_weight(name='total_{}'.format(name), initializer="zeros")
        self.count = self.add_weight(name='count_{}'.format(name), initializer="zeros")

    def update_state(self, result):
        self.total.assign_add(result)
        self.count.assign_add(1)

    def result(self):
        return self.total/self.count

    def reset_state(self):
        # The state of the metric will be reset at the start of each epoch.
        self.total.assign(0.0)
        self.count.assign(0)